In [ ]:
import pandas as pd
import numpy as np
import boto3
from tqdm import tqdm
import os.path as op
import os

In [ ]:
hcp_sub_list = np.loadtxt("./hcp_subs.txt", dtype=int)
s3 = boto3.client('s3')
os.makedirs("prof_csvs", exist_ok=True)
for hcp_sub in tqdm(hcp_sub_list):
    clean_prof_fname = (
        f"sub-{hcp_sub}/ses-01/sub-{hcp_sub}_dwi_space-RASMM_model-CSD"
        "_desc-prob-afq_profiles.csv")
    unclean_prof_fname = (
        f"sub-{hcp_sub}/ses-01/sub-{hcp_sub}_dwi_space-RASMM_model-CSD"
        "_desc-prob-AFQ_profiles.csv")
    if not op.exists(f"prof_csvs/{hcp_sub}.csv"):
        try:
            s3.download_file(
                "open-neurodata",
                f"rokem/hcp1200/afq/{clean_prof_fname}",
                f"prof_csvs/{hcp_sub}.csv")
        except:
            s3.download_file(
                "open-neurodata",
                f"rokem/hcp1200/afq/{unclean_prof_fname}",
                f"prof_csvs/{hcp_sub}.csv")


In [ ]:
if not op.exists("combined_profs.csv"):
    results = pd.DataFrame()
    for hcp_sub in tqdm(hcp_sub_list):
        try:
            this_csv = pd.read_csv(f"prof_csvs/{hcp_sub}.csv")
        except:
            print(f"Sub {hcp_sub} Failed")
        this_csv["subjectID"] = hcp_sub
        results = pd.concat([results, this_csv])
    results.to_csv("combined_profs.csv", index=False)

In [ ]:
import AFQ.viz.altair as AFA

In [ ]:
results = pd.read_csv("combined_profs.csv")
results = AFA.combined_profiles_df_to_altair_df(results, ['dki_fa', 'dki_md', 'dki_mk', 'dki_awf'])
results["Bundle Name"] = results["Bundle Name"].replace({
    "ATR_L": "Anterior Thalamic",
    "ATR_R": "Anterior Thalamic",
    "CST_L": "Corticospinal",
    "CST_R": "Corticospinal",
    "CGC_L": "Cingulum Cingulate",
    "CGC_R": "Cingulum Cingulate",
    "IFO_L": "Inferior Fronto-Occipital",
    "IFO_R": "Inferior Fronto-Occipital",
    "ILF_L": "Inferior Longitudinal",
    "ILF_R": "Inferior Longitudinal",
    "SLF_L": "Superior Longitudinal",
    "SLF_R": "Superior Longitudinal",
    "UNC_L": "Uncinate",
    "UNC_R": "Uncinate",
    "ARC_L": "Arcuate",
    "ARC_R": "Arcuate",
    "VOF_L": "Vertical Occipital",
    "VOF_R": "Vertical Occipital",
    "pARC_L": "Posterior Arcuate",
    "pARC_R": "Posterior Arcuate"
})

In [ ]:
results

In [ ]:
import importlib
importlib.reload(AFA)

In [ ]:
import altair as alt
call_results = results[results.Hemi == "Callosal"]
stand_results = results[results.Hemi != "Callosal"]
prof_chart = AFA.altair_df_to_chart(call_results, font_size=40, line_size=5, color=alt.Color(
            "Bundle Name", 
            scale=alt.Scale(
                domain=[
                    'AntFrontal', 'Motor', 'Occipital', 'Orbital',
                    'PostParietal', 'SupFrontal', 'SupParietal',
                    'Temporal'],
                range=[
                    "rgb(18, 120, 51)", "rgb(136, 205, 238)",
                    "rgb(172, 69, 154)", "rgb(51, 33, 136)",
                    "rgb(205, 102, 120)", "rgb(69, 172, 154)",
                    "rgb(223, 205, 120)", "rgb(136, 33, 84)"])))
prof_chart.save("supp_chart_call.png", dpi=300)
prof_chart = AFA.altair_df_to_chart(stand_results, font_size=40, line_size=5, legend_line_size=3, column_count=2, color="Hemi")
prof_chart.save("supp_chart_stand.png", dpi=300)
prof_chart